In [1]:
from utz import *
from nj_crashes.crashes import Crashes, pk_cols

In [2]:
%%time
c = Crashes.load()
c

CPU times: user 25 s, sys: 11.4 s, total: 36.5 s
Wall time: 37.9 s


,year,cc,mc,case,cn,mn,pdc,pdn,station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24


In [3]:
c[c.duplicated(keep=False, subset=pk_cols)]

,year,cc,mc,case,cn,mn,pdc,pdn,station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [4]:
type(c)

nj_crashes.crashes.Crashes

In [5]:
gdf = c.gdf('o')
gdf

,year,cc,mc,case,cn,mn,pdc,pdn,station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT EMPTY
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07559 40.80707)
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05549 40.84466)
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.07627 40.80564)


### Load county geometries

In [6]:
from nj_crashes.geo import get_boundary_lls, get_county_coords, get_counties, get_county_geometries
from geopandas.tools import sjoin
import geopandas as gpd
from shapely import box
import simplejson
from njdot.codes import CrashSeverity

In [7]:
%%time
cg = get_county_geometries()
cg

Loading /Users/ryan/c/neighbor-ryan/nj-crashes/tlgdb_2022_a_34_nj.gdb/


CPU times: user 135 ms, sys: 34 ms, total: 169 ms
Wall time: 235 ms


,geometry
cn,
Atlantic,"MULTIPOLYGON (((-74.58640 39.30990, -74.58665 ..."
Bergen,"MULTIPOLYGON (((-73.99830 40.80848, -73.99832 ..."
Burlington,"MULTIPOLYGON (((-74.47024 39.92253, -74.47010 ..."
Camden,"MULTIPOLYGON (((-75.01333 39.95420, -75.01332 ..."
Cape May,"MULTIPOLYGON (((-74.54044 39.29945, -74.54012 ..."
Cumberland,"MULTIPOLYGON (((-75.41872 39.43250, -75.41864 ..."
Essex,"MULTIPOLYGON (((-74.13957 40.80541, -74.13959 ..."
Gloucester,"MULTIPOLYGON (((-75.11506 39.86153, -75.11508 ..."
Hudson,"MULTIPOLYGON (((-74.02064 40.73039, -74.02097 ..."


In [9]:
%%time
joined = sjoin(gdf.df[['geometry']], cg)
joined = joined.rename(columns={ 'index_right': 'ocn', })
joined

<timed exec>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4269



CPU times: user 1min 19s, sys: 533 ms, total: 1min 20s
Wall time: 1min 20s


,geometry,ocn
id,,
632693,POINT (-74.49141 39.41477),Atlantic
632919,POINT (-74.49257 39.39608),Atlantic
633737,POINT (-74.44556 39.35505),Atlantic
634226,POINT (-74.43691 39.35858),Atlantic
634419,POINT (-74.45566 39.35287),Atlantic
...,...,...
6077298,POINT (-75.07559 40.80707),Warren
6077299,POINT (-75.05549 40.84466),Warren
6077300,POINT (-75.07627 40.80564),Warren


In [10]:
joined.ocn.value_counts().sort_index()

Atlantic       56825
Bergen        113569
Burlington     72611
Camden         73453
Cape May       13553
Cumberland     30709
Essex         104867
Gloucester     53142
Hudson         35605
Hunterdon      19675
Mercer         49266
Middlesex     117574
Monmouth       98022
Morris         80610
Ocean          80472
Passaic        66918
Salem          11480
Somerset       52014
Sussex         18256
Union          85890
Warren         17135
Name: ocn, dtype: int64

In [11]:
with_ocn = sxs(gdf.df, joined.ocn).sort_index()
with_ocn

,year,cc,mc,case,cn,mn,pdc,pdn,station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry,ocn
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT EMPTY,NaN
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07559 40.80707),Warren
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05549 40.84466),Warren
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.07627 40.80564),Warren


In [12]:
assert with_ocn.index.tolist() == list(range(len(with_ocn)))

In [13]:
ill = Crashes(with_ocn).mp_lls(append=True)
ill

,year,cc,mc,case,cn,mn,pdc,pdn,station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry,ocn,reason,ilon,ilat
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT EMPTY,NaN,NaN,-74.512308,39.432705
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07559 40.80707),Warren,NaN,-75.075607,40.807156
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05549 40.84466),Warren,NaN,-75.056122,40.844345
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.07627 40.80564),Warren,NaN,-75.015149,40.863108


In [14]:
%%time
igdf = Crashes(ill).gdf('i')
igdf

CPU times: user 5.34 s, sys: 8.02 s, total: 13.4 s
Wall time: 16.9 s


,year,cc,mc,case,cn,mn,pdc,pdn,station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry,ocn,reason,ilon,ilat
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT (-74.51231 39.43270),NaN,NaN,-74.512308,39.432705
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07561 40.80716),Warren,NaN,-75.075607,40.807156
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05612 40.84434),Warren,NaN,-75.056122,40.844345
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.01515 40.86311),Warren,NaN,-75.015149,40.863108


In [15]:
type(igdf)

nj_crashes.crashes.Crashes

In [16]:
%%time
ij = sjoin(igdf.df[['geometry']], cg)
ij = ij.rename(columns={ 'index_right': 'icn', })
ij

<timed exec>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4269



CPU times: user 3min 35s, sys: 1.09 s, total: 3min 36s
Wall time: 3min 36s


,geometry,icn
id,,
3,POINT (-74.51231 39.43270),Atlantic
5,POINT (-74.49717 39.42692),Atlantic
6,POINT (-74.50114 39.42445),Atlantic
8,POINT (-74.51094 39.43397),Atlantic
9,POINT (-74.51274 39.43301),Atlantic
...,...,...
6056506,POINT (-74.62165 41.18961),Sussex
6056507,POINT (-74.62955 41.22200),Sussex
6074721,POINT (-74.77471 40.91620),Sussex


In [17]:
with_cns = sxs(igdf.df, ij.icn).sort_index()
with_cns

,year,cc,mc,case,cn,mn,pdc,pdn,station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry,ocn,reason,ilon,ilat,icn
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT (-74.51231 39.43270),NaN,NaN,-74.512308,39.432705,Atlantic
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY,NaN,No MP,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07561 40.80716),Warren,NaN,-75.075607,40.807156,Warren
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05612 40.84434),Warren,NaN,-75.056122,40.844345,Warren
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.01515 40.86311),Warren,NaN,-75.015149,40.863108,Warren


In [18]:
assert with_cns.index.tolist() == list(range(len(with_cns)))

In [19]:
type(with_cns)

geopandas.geodataframe.GeoDataFrame

In [20]:
from nj_crashes.paths import PUBLIC_DIR

In [21]:
path = f'{PUBLIC_DIR}/njdot/with_cns.parquet'

In [22]:
%%time
with_cns.drop(columns='geometry').to_parquet(path)

CPU times: user 11.8 s, sys: 3.49 s, total: 15.2 s
Wall time: 17.9 s


## Export to duckdb (with indices)

In [23]:
import duckdb
dsql = duckdb.sql

In [24]:
duckdb.sql(f"CREATE TABLE crashes AS SELECT * FROM '{path}';")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [25]:
dsql("SHOW TABLES")

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ crashes │
└─────────┘

In [26]:
%%time
dsql(f"CREATE UNIQUE INDEX crashes_id_idx ON crashes (id);")

CPU times: user 848 ms, sys: 361 ms, total: 1.21 s
Wall time: 311 ms


In [27]:
%%time
dsql(f"CREATE INDEX crashes_dt_icn_severity_idx ON crashes (dt, icn, severity);")

CPU times: user 1.82 s, sys: 56.3 ms, total: 1.88 s
Wall time: 572 ms


In [28]:
%%time
dsql("EXPORT DATABASE 'crashes.ddb' (FORMAT PARQUET)")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 20.4 s, sys: 4.35 s, total: 24.8 s
Wall time: 5.82 s


In [103]:
dsql("SELECT * from duckdb_indexes()")

┌───────────────┬──────────────┬─────────────┬────────────┬───┬────────────┬─────────────┬──────────────────────┐
│ database_name │ database_oid │ schema_name │ schema_oid │ … │ is_primary │ expressions │         sql          │
│    varchar    │    int64     │   varchar   │   int64    │   │  boolean   │   varchar   │       varchar        │
├───────────────┼──────────────┼─────────────┼────────────┼───┼────────────┼─────────────┼──────────────────────┤
│ memory        │          992 │ main        │        994 │ … │ false      │ NULL        │ CREATE UNIQUE INDE…  │
├───────────────┴──────────────┴─────────────┴────────────┴───┴────────────┴─────────────┴──────────────────────┤
│ 1 rows                                                                                   12 columns (7 shown) │
└───────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [93]:
dsql("SELECT * FROM crashes WHERE id=1000000")

┌───────┬───────┬───────┬─────────┬─────────┬───┬─────────┬─────────┬────────┬────────┬─────────┬─────────┐
│ year  │  cc   │  mc   │  case   │   cn    │ … │   ocn   │ reason  │  ilon  │  ilat  │   icn   │   id    │
│ int64 │ int64 │ int64 │ varchar │ varchar │   │ varchar │ varchar │ double │ double │ varchar │  int64  │
├───────┼───────┼───────┼─────────┼─────────┼───┼─────────┼─────────┼────────┼────────┼─────────┼─────────┤
│  2004 │     2 │    63 │ 04-077  │ Bergen  │ … │ NULL    │ No MP   │   NULL │   NULL │ NULL    │ 1000000 │
├───────┴───────┴───────┴─────────┴─────────┴───┴─────────┴─────────┴────────┴────────┴─────────┴─────────┤
│ 1 rows                                                                            57 columns (11 shown) │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [30]:
dsql("DESCRIBE crashes")

┌───────────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│        column_name        │ column_type │  null   │   key   │ default │ extra │
│          varchar          │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ year                      │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ cc                        │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ mc                        │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ case                      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ cn                        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ mn                        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ pdc                       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ pdn                       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ station       

In [40]:
%%time
dsql("""
SELECT date_part('year', dt) as year, icn as county, severity, count(*) as num
FROM crashes
WHERE icn IS NOT NULL
GROUP BY (date_part('year', dt), icn, severity)
ORDER BY (date_part('year', dt), icn, severity)
""")

CPU times: user 771 µs, sys: 26 µs, total: 797 µs
Wall time: 809 µs


┌───────┬────────────┬──────────┬───────┐
│ year  │   county   │ severity │  num  │
│ int64 │  varchar   │ varchar  │ int64 │
├───────┼────────────┼──────────┼───────┤
│  2001 │ Atlantic   │ f        │    49 │
│  2001 │ Atlantic   │ i        │  1689 │
│  2001 │ Atlantic   │ p        │  2860 │
│  2001 │ Bergen     │ f        │    16 │
│  2001 │ Bergen     │ i        │  3181 │
│  2001 │ Bergen     │ p        │  7189 │
│  2001 │ Burlington │ f        │    54 │
│  2001 │ Burlington │ i        │  1829 │
│  2001 │ Burlington │ p        │  4050 │
│  2001 │ Camden     │ f        │    33 │
│    ·  │   ·        │ ·        │     · │
│    ·  │   ·        │ ·        │     · │
│    ·  │   ·        │ ·        │     · │
│  2021 │ Somerset   │ p        │  5506 │
│  2021 │ Sussex     │ f        │     9 │
│  2021 │ Sussex     │ i        │   423 │
│  2021 │ Sussex     │ p        │  1591 │
│  2021 │ Union      │ f        │    28 │
│  2021 │ Union      │ i        │  3698 │
│  2021 │ Union      │ p        │ 

In [99]:
dsql("EXPLAIN SELECT year, count(*) as num FROM crashes GROUP BY year")

┌───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  explain_key  │                                            explain_value                                             │
│    varchar    │                                               varchar                                                │
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ physical_plan │ ┌───────────────────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│__in…  │
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [97]:
dsql("EXPLAIN ANALYZE SELECT * FROM crashes WHERE id=1000000")

┌───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  explain_key  │                                            explain_value                                             │
│    varchar    │                                               varchar                                                │
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ analyzed_plan │ ┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││    Query Prof…  │
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────┘



## Check county names
- `cn`: `County Name` value
- `ocn`: county name implied by "original" `Latitude`/`Longitude` fields (renamed `olat`/`olon` here)
- `icn`: county name implied by "interpolated" lat/lon (`ilat`/`ilon`) value

In [39]:
cn_ocn = with_cns.ocn.copy().rename('cn_ocn').fillna('nan')
ocn_isn = ~with_cns.ocn.isna()
cn_ocn.loc[ocn_isn & (with_cns.ocn == with_cns.cn)] = 'eq'
cn_ocn.loc[ocn_isn & (with_cns.ocn != with_cns.cn)] = 'neq'

In [40]:
cn_icn = with_cns.icn.copy().rename('cn_icn').fillna('nan')
icn_isn = ~with_cns.icn.isna()
cn_icn.loc[icn_isn & (with_cns.icn == with_cns.cn)] = 'eq'
cn_icn.loc[icn_isn & (with_cns.icn != with_cns.cn)] = 'neq'

In [61]:
names = [ 'nan', 'eq', 'neq' ]
ct = pd.crosstab(cn_ocn, cn_icn).loc[names, names]
n = ct.sum().sum()
ct.loc['sum'] = ct.sum(0)
ct['sum'] = ct.sum(1)
ct

cn_icn,nan,eq,neq,sum
cn_ocn,,,,
nan,2505195,2297595,22867,4825657
eq,90888,1134018,5381,1230287
neq,1478,15327,4554,21359
sum,2597561,3446940,32802,6077303


In [62]:
pct = round(ct / n * 100, 1)
pct

cn_icn,nan,eq,neq,sum
cn_ocn,,,,
nan,41.2,37.8,0.4,79.4
eq,1.5,18.7,0.1,20.2
neq,0.0,0.3,0.1,0.4
sum,42.7,56.7,0.5,100.0


In crashes where:
- "original lat/lon" and and "interpolated lat/lon" county names both exist, and
- both don't match the provided `County Name`,

Do they at least match each other?

In [64]:
neqs = with_cns[(cn_icn == 'neq') & (cn_ocn == 'neq')]
neqs_eq = neqs.icn == neqs.ocn
neqs_eq.value_counts()

True     4445
False     109
dtype: int64

Crashes with 3 different county names / locations 😵‍💫

In [69]:
with_cns[['cn', 'icn', 'ocn']].loc[neqs_eq[~neqs_eq].index].value_counts().sort_index()

cn          icn         ocn       
Bergen      Hudson      Essex          2
            Passaic     Essex          1
                        Monmouth       1
Burlington  Camden      Cumberland     1
                        Gloucester     1
                        Ocean          1
Camden      Burlington  Gloucester    10
                        Salem          2
            Gloucester  Atlantic       2
                        Cumberland     1
                        Salem          1
Cape May    Atlantic    Burlington     1
                        Cumberland     2
                        Ocean          1
Cumberland  Salem       Gloucester     2
Essex       Hudson      Hunterdon      1
                        Ocean          1
            Morris      Somerset       1
                        Union          1
            Union       Middlesex      3
                        Morris        12
                        Somerset       2
Gloucester  Camden      Atlantic       7
                      

---

## Misc crash stats

In [5]:
c['Crash Type Code'].value_counts().sort_index()

         3529
00        276
01    1671264
02     776875
03     856433
04     100301
05      42280
06     791254
07     166736
08     336399
09      14009
10      33381
11     738090
12     180445
13     102632
14      44236
15      58085
16        619
20      28957
99     131502
Name: Crash Type Code, dtype: int64

In [6]:
c['Unit Of Measurement'].value_counts()

FE    2426156
      1853661
AT    1742308
MI      55178
Name: Unit Of Measurement, dtype: int64

In [10]:
(c.mp == 0).sum()

76186

In [9]:
c.isna().sum()

County Name                             0
Municipality Name                       0
Police Department Code                  0
Police Department                       0
Police Station                          0
Total Killed                            0
Total Injured                           0
Pedestrians Killed                      0
Pedestrians Injured                     0
severity                                0
Intersection                            0
Alcohol Involved                        0
HazMat Involved                         0
Crash Type Code                         0
Total Vehicles Involved                 0
Crash Location                          0
Location Direction                      0
Route                                   0
Route Suffix                            0
sri                                     0
mp                                2367547
Road System                             0
Road Character                          0
Road Surface Type                 